In [1]:
import tensorflow as tf
import tensorboard as tb
from google.protobuf import json_format

# Work in python dir to import protobuf generated python module for eval out

%cd '/Users/slayton/paradox/user-protection/up-ml-models/user-protection-pipeline/tf-supervised/src/main/python'
from trainers.schemas import eval_out_pb2
reload(eval_out_pb2)

/Users/slayton/paradox/user-protection/up-ml-models/user-protection-pipeline/tf-supervised/src/main/python


<module 'trainers.schemas.eval_out_pb2' from 'trainers/schemas/eval_out_pb2.pyc'>

In [ ]:
event = "/Users/slayton/Downloads/email_tf_training_Email.unsubscribed.Train.DefaultFeatures.LinearModel_2019-04-28_20190621T180108.458687-1179a67ed6df_eval_LinearModelMetrics_events.out.tfevents.1561140225.cmle-training-master-eb7659c54d-0-w5l9n"



In [ ]:
[json_format.MessageToDict(x).keys() for x in tf.train.summary_iterator(event)]

In [ ]:
for summary in tf.train.summary_iterator(event):
    for v in summary.summary.value:
        print(v.tag)
                                         
                                         
                                         
                                         
                                         

In [ ]:
from tensorboard.plugins.pr_curve import pr_curves_plugin
from tensorboard.util import tensor_util
from tensorboard.plugins.pr_curve import metadata

In [ ]:
[e for e in [x for x in tf.train.summary_iterator(event)]]
 

In [ ]:
def compute_thresholds(num_thresholds):
    """Computes a list of specific thresholds from the number of thresholds.
    Args:
      num_thresholds: The number of thresholds.
    Returns:
      A list of specific thresholds (floats).
    """
    return [float(v) / num_thresholds for v in range(1, num_thresholds + 1)]
    

def _process_tensor_event(v, thresholds, tag):
    """Converts a TensorEvent into a dict that encapsulates information on it.
    Args:
      event: The TensorEvent to convert.
      thresholds: An array of floats that ranges from 0 to 1 (in that
        direction and inclusive of 0 and 1).
    Returns:
      A JSON-able dictionary of PR curve data for 1 step.
    """
    pr = None
    for m in v.summary.value:
        if m.tag == tag:
            tnsr = m.tensor
            pr = _make_pr_entry(
                                tensor_util.make_ndarray(tnsr),
                                compute_thresholds(thresholds))
        else:
            pr = pr
    
    return pr


def _make_pr_entry(data_array, thresholds):
    """Creates an entry for PR curve data. Each entry corresponds to 1 step.
    Args:
      step: The step.
      wall_time: The wall time.
      data_array: A numpy array of PR curve data stored in the summary format.
      thresholds: An array of floating point thresholds.
    Returns:
      A PR curve entry.
    """
    # Trim entries for which TP + FP = 0 (precision is undefined) at the tail of
    # the data.
    true_positives = [int(v) for v in data_array[metadata.TRUE_POSITIVES_INDEX]]
    false_positives = [
        int(v) for v in data_array[metadata.FALSE_POSITIVES_INDEX]]
    tp_index = metadata.TRUE_POSITIVES_INDEX
    fp_index = metadata.FALSE_POSITIVES_INDEX
    positives = data_array[[tp_index, fp_index], :].astype(int).sum(axis=0)
    end_index_inclusive = len(positives) - 1
    while end_index_inclusive > 0 and positives[end_index_inclusive] == 0:
      end_index_inclusive -= 1
    end_index = end_index_inclusive + 1

    return {
        'precision': data_array[metadata.PRECISION_INDEX, :end_index].tolist(),
        'recall': data_array[metadata.RECALL_INDEX, :end_index].tolist(),
        'true_positives': true_positives[:end_index],
        'false_positives': false_positives[:end_index],
        'true_negatives':
            [int(v) for v in
             data_array[metadata.TRUE_NEGATIVES_INDEX][:end_index]],
        'false_negatives':
            [int(v) for v in
             data_array[metadata.FALSE_NEGATIVES_INDEX][:end_index]],
        'thresholds': thresholds[:end_index],
    }

def loop_events(event_file, num_thresholds):
    output = []
    for e in [x for x in tf.train.summary_iterator(event)]:
        for v in e.summary.value:
            if v.tensor.dtype != 0:
                if v.tensor.tensor_shape != 0:
                    if len(v.tensor.tensor_shape.dim) > 1:
                        if v.tensor.tensor_shape.dim[0] != v.tensor.tensor_shape.dim[1]:
                        # Curve Data
                            print(_process_tensor_event(e, num_thresholds, v.tag))
                            output.append(_process_tensor_event(e, num_thresholds, v.tag))
            else:
                print(v)
                output.append({'tag':v.tag, 'value': v.simple_value, 'step': e.step,'wall_time': e.wall_time})
    return output
                    
    

In [ ]:
[x for x in tf.train.summary_iterator(event)]



In [ ]:
pr_curves = loop_events(event, 21)

In [ ]:
import pandas as pd
pd.DataFrame(pr_curves)

In [ ]:
from luigi.contrib.gcs import GCSClient
from os.path import join as pjoin
from os.path import basename

In [ ]:
# Function to convert from tfEvent to Eval Out Proto
def event_to_eval_out(event_path, thresholds):
    # Initialize Output List
    eval_out = []
    # Parse Event File
    events = tf.train.summary_iterator(event_path)
    for e in events:
        #print(e.summary)
        if len(e.summary.value)>0:
            eval_message = eval_out_pb2.Eval()
            eval_metrics = eval_message.eval_metrics.add()
            eval_metrics.global_step = e.step
            for v in e.summary.value:
                set_proto_value(v, eval_metrics, thresholds)
            eval_out.append(eval_message)
    return eval_out

def set_proto_value(v, message, thresholds):
    try:
        # - Assumes name in eval_out proto == name in Model.py eval_config_metrics dict
        # Strip /0 appended by pr_curve and or_curve plugin code that we don't own
        getattr(message, ''.join([i for i in v.tag if (i != "0") and (i != "/")]))
        
        if "/0" in v.tag:
            # special handling for pr and or curves
            process_curves(v, message, thresholds)
        else:
            setattr(message, v.tag, v.simple_value)
    except AttributeError:
        pass

            
def precision(true_pos, false_pos):
    return float(true_pos / (false_pos + true_pos))

def recall(true_pos, false_neg):
    return float(true_pos / (true_pos + false_neg))
    
# Ugly function for PR Curve Plugin Data
def process_curves(v, eval_metrics, thresholds):
    tnsr = v.tensor
    pr = _make_pr_entry(
                tensor_util.make_ndarray(tnsr),
                compute_thresholds(thresholds))
    for i, thresh in enumerate(pr['thresholds']):
        if v.tag == 'or_curve/0':
            or_curve = eval_metrics.or_curve.add()
            or_curve.open_rate = float(pr['precision'][i])
            or_curve.reach_rate = float(pr['recall'][i])
        if v.tag == 'pr_curve/0':
            or_curve = eval_metrics.pr_curve.add()
            or_curve.precision = float(pr['precision'][i])
            or_curve.recall = float(pr['recall'][i])            
        or_curve.threshold = float(pr['thresholds'][i])
        or_curve.true_positive = float(pr['true_positives'][i])
        or_curve.true_negative = float(pr['true_negatives'][i])
        or_curve.false_positive = float(pr['false_positives'][i])
        or_curve.false_negative = float(pr['false_negatives'][i])

def get_eval_file(jobdir, estimator_type):
    # jobdir is the path set as the training job dir in the training job
    eval_path = pjoin(jobdir, "eval_{estimator_type}Metrics".format(estimator_type = estimator_type))
    client = GCSClient()
    eval_files = client.listdir(eval_path)
    #print(eval_path)
    out_times = []
    for f in eval_files:
        if f.strip("/") != eval_path:
            # Get Write Times
            # Assumes filename form `"events"."out"."tfevents".{Timestamp}...`
            # Which is set by tensorflow and could break on update
            out_times.append((f, int(basename(f).split(".")[3])))
    # Get max file path
    print(max(out_times,key=lambda item:item[1])[0])
    return max(out_times,key=lambda item:item[1])[0]



In [ ]:
jobdir = "gs://up_ml/email/tf/training/Email.unsubscribed.Train.DefaultFeatures.LinearModel/2019-04-28/20190621T180108.458687-1179a67ed6df"
estimator_type = "LinearModel"

In [ ]:
metrics = event_to_eval_out(get_eval_file(jobdir, estimator_type), 21)

In [ ]:
json_format.MessageToDict(metrics[0],preserving_proto_field_name=True)

In [ ]:
# Let's Write to BQ
import json
import sys

from google.cloud import bigquery

In [ ]:
project = 'users-protection'
dataset_id = "slayton_test"  # the ID of your dataset
table_id = "test_bq_write"  # the ID of your table

In [ ]:
# Read Schema function
def get_tbl_schema():
    with open('trainers/schemas/eval_out/eval_table.schema') as f:
        d = json.load(f)
        return d
        
schema = get_tbl_schema()


def _get_field_schema(field):
    name = field['name']
    field_type = field.get('type', 'STRING')
    mode = field.get('mode', 'NULLABLE')
    fields = field.get('fields', [])

    if fields:
        subschema = []
        for f in fields:
            fields_res = _get_field_schema(f)
            subschema.append(fields_res)
    else:
        subschema = []

    field_schema = bigquery.SchemaField(name=name, 
        field_type=field_type,
        mode=mode,
        fields=subschema
    )
    return field_schema


def parse_bq_json_schema(schema_filename):
    schema = []
    with open(schema_filename, 'r') as infile:
        jsonschema = json.load(infile)

    for field in jsonschema:
        schema.append(_get_field_schema(field))

    return schema

parse_bq_json_schema('trainers/schemas/eval_out/eval_table.schema')


In [ ]:
def write_to_bq(project, dataset, table):
    client = bigquery.Client(project=project)
    table_ref = client.dataset(dataset_id).table(table_id)
    try:
        # Check if table exists if does, yay
        table = client.get_table(table_ref)
    except:
        # Create Table
        table = bigquery.Table(table_ref, schema=parse_bq_json_schema('trainers/schemas/eval_out/eval_table.schema'))
        table = client.create_table(table)  # API request
        assert table.table_id == table_id

    client.insert_rows_json(table, [json_format.MessageToDict(metrics[0], preserving_proto_field_name=True)])

In [ ]:
from trainers.schemas import eval_out_pb2
from trainers.schemas import train_task_params_pb2



eval_out_message = eval_out_pb2.EvalOut()
eval_message = eval_out_pb2.Eval()
train_param_message = train_task_params_pb2.TrainInstance()
eval_out_message.model.train_instance.CopyFrom(train_param_message)
eval_out_message.eval.CopyFrom(eval_message)



In [2]:
jobdir = "gs://up_ml/email/tf/training/Email.unsubscribed.Train.DefaultFeatures.LinearModel/2019-04-28/20190621T180108.458687-1179a67ed6df"
estimator_type = "LinearModel"

from tasks.eval_out_write import BQEval, BQWriter
eval_out = BQEval(jobdir, 21, estimator_type).run()[0]

eval_out

/Users/slayton/virtual_envs/tensorflow_env/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


gs://up_ml/email/tf/training/Email.unsubscribed.Train.DefaultFeatures.LinearModel/2019-04-28/20190621T180108.458687-1179a67ed6df/eval_LinearModelMetrics/events.out.tfevents.1561140225.cmle-training-master-eb7659c54d-0-w5l9n
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Ignoring Eval Type: loss
Ignoring Eval Type: confusion_matrix
Ignoring Eval Type: checkpoint_path


eval_metrics {
  global_step: 10004
  accuracy: 0.898695528507
  auc: 0.537977695465
  precision: 0.00110558315646
  recall: 0.124352328479
  f1_score: 0.00219167885371
  true_pos: 24.0
  or_curve {
    threshold: 0.047619048506
    true_positive: 193.0
    true_negative: 0.0
    false_positive: 215523.0
    false_negative: 0.0
    open_rate: 0.000894694880117
    reach_rate: 1.0
  }
  or_curve {
    threshold: 0.095238097012
    true_positive: 193.0
    true_negative: 1079.0
    false_positive: 214444.0
    false_negative: 0.0
    open_rate: 0.000899192586076
    reach_rate: 0.994998037815
  }
  or_curve {
    threshold: 0.142857149243
    true_positive: 187.0
    true_negative: 7507.0
    false_positive: 208016.0
    false_negative: 6.0
    open_rate: 0.000898161903024
    reach_rate: 0.965171813965
  }
  or_curve {
    threshold: 0.190476194024
    true_positive: 176.0
    true_negative: 31668.0
    false_positive: 183855.0
    false_negative: 17.0
    open_rate: 0.000956360599957
 

In [3]:
BQWriter(data=eval_out, project="users-protection",dataset="slayton_test2", table="eval_out_test", table_schema="trainers/schemas/eval_out/eval_out_table.schema").write_to_bq()





writing rows
